In [57]:
import tensorflow as tf
import numpy as np
import pandas as pd

np.set_printoptions(precision=3, suppress=True)
data_train=pd.read_csv("train.csv")
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders','delivered_time'],axis=1)
labels=list(data_train.head(0))
x_id=data_train["order_id"]
y=data_train["cancelled"]
data_train=data_train.drop(["cancelled","order_id"],axis=1)

In [58]:
data_train.head()

,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,lifetime_order_count,session_time
0,1.5666,2.65,46.0,46.0,621.0,NaN
1,2.5207,2.76,8.0,8.0,105.0,3.266667
2,2.2074,4.80,1.0,1.0,66.0,9.816667
3,2.1894,6.38,1.0,1.0,127.0,17.533333
4,2.7870,4.01,34.0,34.0,84.0,1.350000


In [59]:
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
data_train.shape


(450000, 6)

In [60]:
y.to_numpy()
train_examples=data_train
train_labels=y


In [61]:
data_test=pd.read_csv("test.csv")
data_test=data_test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason','reassigned_order','rider_id','reassignment_method','undelivered_orders'],axis=1)
labels=list(data_test.head(0))
test_id=data_test["order_id"]
y=data_test["cancelled"]
data_test=data_test.drop(["cancelled","order_id"],axis=1)
data_test.to_numpy()
data_test=np.nan_to_num(data_test)
y.to_numpy()
test_examples=data_test
test_labels=y

In [62]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 450
tf.convert_to_tensor(train_examples)

<tf.Tensor: shape=(450000, 6), dtype=float64, numpy=
array([[   1.567,    2.65 ,   46.   ,   46.   ,  621.   ,    0.   ],
       [   2.521,    2.76 ,    8.   ,    8.   ,  105.   ,    3.267],
       [   2.207,    4.8  ,    1.   ,    1.   ,   66.   ,    9.817],
       ...,
       [   1.594,    1.61 ,   28.   ,   28.   , 1488.   ,  204.15 ],
       [   2.894,    4.68 ,   72.   ,   72.   ,  105.   ,   65.583],
       [   1.893,    0.09 ,   30.   ,   30.   ,  108.   ,  212.   ]])>

In [63]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_examples)

2022-02-02 01:25:52.031771: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 01:25:52.042832: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [83]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  opt = tf.keras.optimizers.SGD(lr=0.01)
  model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
'''model = get_basic_model()
model.fit(train_examples, train_labels, epochs=15, batch_size=BATCH_SIZE)'''

In [84]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
train_batches = train_dataset.shuffle(1000).batch(450)

model = get_basic_model()
model.fit(train_batches, epochs=15)

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-02 01:43:23.422036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/15
1000/1000 [==============================] - 8s 7ms/step - loss: 0.2778 - accuracy: 0.9855
Epoch 2/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1102 - accuracy: 0.9884
Epoch 3/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0844 - accuracy: 0.9884
Epoch 4/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0751 - accuracy: 0.9884
Epoch 5/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0705 - accuracy: 0.9884
Epoch 6/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0680 - accuracy: 0.9884
Epoch 7/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0665 - accuracy: 0.9884
Epoch 8/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0655 - accuracy: 0.9884
Epoch 9/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0649 - accuracy: 0.9884
Epoch 10/15
1000/1000 [==============================] - 7s 7ms/step - loss: 0.064

In [85]:
tf.convert_to_tensor(test_examples)
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
model.evaluate(test_examples,test_labels)

  30/4527 [..............................] - ETA: 15s - loss: 0.0143 - accuracy: 1.0000

2022-02-02 01:45:13.973169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


4527/4527 [==============================] - 15s 3ms/step - loss: 0.0143 - accuracy: 1.0000


[0.014297780580818653, 1.0]

In [120]:
ans=model.predict(test_examples)

In [121]:
ans.shape

(144844, 1)

In [130]:
ans=list(ans)

In [142]:
min(ans)

array([0.014], dtype=float32)

In [116]:
ans.reshape(-1)
for i in range(0,144844):
    if(ans[i] == 0.014):
        ans[i]=0
    else:
        ans[i]=1

In [119]:
min(ans)

array([1.], dtype=float32)

In [97]:
df = pd. DataFrame([[test_id], [ans]],columns=["order_id", "cancelled"])

ValueError: 2 columns passed, passed data had 1 columns